<a href="https://colab.research.google.com/github/IverMartinsen/deeplearning/blob/master/problem3c.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#@title import modules { form-width: "20%" }
import sys
from google.colab import drive

drive.mount('/content/gdrive')
sys.path.append('/content/gdrive/MyDrive/DL home exam/Problem 3/problem3/')

import tensorflow as tf
from tensorflow.keras.layers import Conv2D, MaxPool2D, Dense
from tensorflow.keras.layers import BatchNormalization, Flatten, Reshape
from tensorflow.keras import Model
import numpy as np
from deshuffler import deshuffler
from load_data import load_data
from sklearn.neighbors import KNeighborsClassifier as KNN

In [ ]:
#@title load data { form-width: "20%" }
x_tr, y_tr, x_te, y_te = load_data(
    '/content/gdrive/MyDrive/DL home exam/Problem 3/problem3/DataNormal.npz')

x_tr = np.moveaxis(x_tr, 1, -1) / 255
x_te = np.moveaxis(x_te, 1, -1) / 255

num_class = len(np.unique(y_tr))

In [ ]:
#@title define models { form-width: "20%" }
class CNN(Model):
    def __init__(self):
        super().__init__()

        self.conv1 = Conv2D(64, 3, padding='same', activation='relu')
        self.norm1 = BatchNormalization()
        self.pool1 = MaxPool2D()
        
        self.conv2 = Conv2D(128, 3, padding='same', activation='relu')
        self.norm2 = BatchNormalization()
        self.pool2 = MaxPool2D()
        
        self.conv3 = Conv2D(256, 3, padding='same', activation='relu')
        self.norm3 = BatchNormalization()
        self.conv4 = Conv2D(256, 3, padding='same', activation='relu')
        self.norm4 = BatchNormalization()
        self.pool3 = MaxPool2D()
        
        self.conv5 = Conv2D(512, 3, padding='same', activation='relu')
        self.norm5 = BatchNormalization()
        self.conv6 = Conv2D(512, 3, padding='same', activation='relu')
        self.norm6 = BatchNormalization()
        self.pool4 = MaxPool2D()
        
        self.conv7 = Conv2D(512, 3, padding='same', activation='relu')
        self.norm7 = BatchNormalization()
        self.conv8 = Conv2D(512, 3, padding='same', activation='relu')
        self.norm8 = BatchNormalization()
        self.pool5 = MaxPool2D()
        
        self.flatten = Flatten()

        self.dense1 = Dense(4096,
                            activation='relu',
                            kernel_regularizer=tf.keras.regularizers.L2(),
                            input_shape=(512, ))
        self.dense2 = Dense(4096,
                            activation='relu',
                            kernel_regularizer=tf.keras.regularizers.L2())
        self.dense3 = Dense(1000,
                            activation='relu',
                            kernel_regularizer=tf.keras.regularizers.L2())

    def call(self, images):
        x = self.pool1(self.norm1(self.conv1(images)))

        x = self.pool2(self.norm2(self.conv2(x)))

        x = self.pool3(self.norm4(self.conv4(self.norm3(self.conv3(x)))))

        x = self.pool4(self.norm6(self.conv6(self.norm5(self.conv5(x)))))

        x = self.pool5(self.norm8(self.conv8(self.norm7(self.conv7(x)))))

        return self.dense3(self.dense2(self.dense1(self.flatten(x))))

In [ ]:
#@title knn on extracted features { form-width: "20%" }
#model = Puzzle_solver()
model = CNN()

model.load_weights('/content/gdrive/MyDrive/DL home exam/Problem 3/ckpt_cn')

model.compile()

batch_size = 25
x_tr = tf.image.resize(x_tr, (48, 48))
x_tr = np.vstack([model(x_tr[batch_size*i:batch_size*(i + 1)]).numpy() \
                  for i in range(x_tr.shape[0] // batch_size)])

x_te = tf.image.resize(x_te, (48, 48))
x_te = np.vstack([model(x_te[batch_size*i:batch_size*(i + 1)]).numpy() \
                  for i in range(x_te.shape[0] // batch_size)])

knn = KNN(3)

knn.fit(x_tr, y_tr)

accuracy = np.sum(knn.predict(x_te) == y_te) / y_te.shape

In [ ]:
#@title knn on extracted features, untrained { form-width: "20%" }
cnn = CNN()
cnn.compile()

batch_size = 25

x_tr = tf.image.resize(x_tr, (48, 48))
x_tr = np.vstack([cnn(x_tr[batch_size*i:batch_size*(i + 1)]).numpy() \
                  for i in range(x_tr.shape[0] // batch_size)])

x_te = tf.image.resize(x_te, (48, 48))
x_te = np.vstack([cnn(x_te[batch_size*i:batch_size*(i + 1)]).numpy() \
                  for i in range(x_te.shape[0] // batch_size)])

knn = KNN(3)

knn.fit(x_tr, y_tr)

accuracy = np.sum(knn.predict(x_te) == y_te) / y_te.shape

In [ ]:
#@title knn on original input { form-width: "20%" }
knn = KNN(3)

knn.fit(x_tr.reshape(x_tr.shape[0], -1), y_tr)

accuracy = np.sum(knn.predict(x_te.reshape(x_te.shape[0], -1) == y_te) / y_te.shape